In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text  # Importar text
import os
from dotenv import load_dotenv
import time

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))  # Asegúrate de convertir a entero
db_name = os.getenv('DB_NAME')

# Función para conectar a la base de datos
def connect_to_db():
    try:
        engine = sqlalchemy.create_engine(
            f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}',
            connect_args={'connect_timeout': 30}  # Ajustar el tiempo de espera de conexión
        )
        connection = engine.connect()
        print("Conexión a la base de datos establecida con éxito.")
        return engine, connection
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        raise

# Conectar a MySQL utilizando las credenciales del archivo .env
engine, connection = connect_to_db()

# Consulta para eliminar los registros deseados
try:
    # Definir las fechas de inicio y fin del mes deseado
    start_date = '2024-07-01 00:00:00'
    end_date = '2024-07-31 23:59:59'

    # Verificar cuántos registros serán eliminados antes de ejecutar el DELETE
    select_query = text("""
        SELECT COUNT(*) FROM taxi_fhv_data
        WHERE source LIKE :source_pattern
        AND Pickup_datetime BETWEEN :start_date AND :end_date;
    """)
    result = connection.execute(select_query, {"source_pattern": "yellow%", "start_date": start_date, "end_date": end_date})
    count = result.fetchone()[0]
    print(f"Registros que se eliminarán: {count}")

    # Confirmación antes de eliminar
    confirm = input("¿Deseas continuar con la eliminación? (sí/no): ").strip().lower()
    if confirm == 'si':
        delete_query = text("""
            DELETE FROM taxi_fhv_data
            WHERE source LIKE :source_pattern
            AND Pickup_datetime BETWEEN :start_date AND :end_date;
        """)
        result = connection.execute(delete_query, {"source_pattern": "yellow%", "start_date": start_date, "end_date": end_date})
        print(f"Registros eliminados: {result.rowcount}")
    else:
        print("Operación cancelada por el usuario.")

except Exception as e:
    print(f"Error durante la ejecución de la consulta: {e}")
finally:
    connection.close()
    engine.dispose()


Conexión a la base de datos establecida con éxito.
Registros que se eliminarán: 1499986
Registros eliminados: 1499986


In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))  # Asegúrate de convertir a entero
db_name = os.getenv('DB_NAME')

# Función para conectar a la base de datos
def connect_to_db():
    try:
        engine = sqlalchemy.create_engine(
            f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}',
            connect_args={'connect_timeout': 30}  # Ajustar el tiempo de espera de conexión
        )
        connection = engine.connect()
        print("Conexión a la base de datos establecida con éxito.")
        return engine, connection
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        raise

# Conectar a MySQL utilizando las credenciales del archivo .env
engine, connection = connect_to_db()

# Crear los índices necesarios
try:
    create_index_source = text("CREATE INDEX idx_source ON taxi_fhv_data(source);")
    connection.execute(create_index_source)
    print("Índice idx_source creado con éxito.")

    create_index_pickup_datetime = text("CREATE INDEX idx_pickup_datetime ON taxi_fhv_data(Pickup_datetime);")
    connection.execute(create_index_pickup_datetime)
    print("Índice idx_pickup_datetime creado con éxito.")
except Exception as e:
    print(f"Error al crear los índices: {e}")

# Analizar y Optimizar Consultas con EXPLAIN
try:
    explain_query = text("""
        EXPLAIN SELECT DISTINCT DATE_FORMAT(Pickup_datetime, '%Y-%m') AS month
        FROM taxi_fhv_data
        WHERE source = 'yellow_tripdata'
        ORDER BY month;
    """)
    result = connection.execute(explain_query)
    explain_plan = result.fetchall()
    print("Plan de ejecución de la consulta:")
    for row in explain_plan:
        print(row)
except Exception as e:
    print(f"Error al analizar la consulta: {e}")

# Ejecutar la consulta optimizada
try:
    query = text("""
        SELECT DISTINCT DATE_FORMAT(Pickup_datetime, '%Y-%m') AS month
        FROM taxi_fhv_data
        WHERE source = 'yellow_tripdata'
        ORDER BY month;
    """)
    result = connection.execute(query)
    months = [row[0] for row in result]  # Acceder al primer elemento de la tupla
    print("Meses disponibles de 'yellow_tripdata':")
    for month in months:
        print(month)
except Exception as e:
    print(f"Error durante la ejecución de la consulta: {e}")
finally:
    connection.close()
    engine.dispose()


Conexión a la base de datos establecida con éxito.
Índice idx_source creado con éxito.
